In [3]:
import sys
import os
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.join(PROJ_DIR,'src'))
import xai_faithfulness_experiments_lib_edits as fl

DICT_PATH_TRAIN = os.path.join(PROJ_DIR, 'data', 'cmnist_train_dict.pickle')
DICT_PATH_TEST = os.path.join(PROJ_DIR, 'data', 'cmnist_test_dict.pickle')

In [6]:
data[0].keys()

dict_keys(['id', 'x', 'y', 's_box', 's_digit', 's_area'])

In [4]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')
BATCH_SIZE_TRAIN = 256
BATCH_SIZE_TEST = 256

model = fl.load_pretrained_cmnist_resnet18_model(os.path.join(PROJ_DIR,'assets','models','cmnist-resnet18.pth'))

Using cuda:0


In [8]:
test_loader = fl.get_cmnist_test_loader(DICT_PATH_TEST, BATCH_SIZE_TRAIN)
test_hits = 0
num_elems = 0
for x_batch, y_batch in test_loader:
    x_batch =  x_batch.to(device)
    y_batch =  y_batch.to(device)
    test_preds = model.forward(x_batch)        
    test_hits += (test_preds.argmax(dim=1) == y_batch).float().sum()
    num_elems += y_batch.shape[0]
print(test_hits / num_elems)

tensor(0.8207, device='cuda:0')


In [21]:
import numpy as np
import captum_generator as cg
from quantus import PointingGame
test_set = fl.CMNISTDataset(dict_file_path=DICT_PATH_TEST)
mask_name = 's_area'

for elem in test_set.data.values():
    x = torch.tensor(elem['x']).to(device)
    y = torch.tensor(elem['y']).to(device)
    rankings = cg.generate_rankings(x, y, model)
    pg = PointingGame(disable_warnings=True)
    x_batch = x.unsqueeze(0).detach().cpu().numpy()
    y_batch = y.unsqueeze(0).detach().cpu().numpy()
    s_batch = np.expand_dims(elem[mask_name].sum(axis=0, keepdims=True), axis = 0)
    print(s_batch.shape)
    for r in rankings:
        score = pg(model, \
                    x_batch, \
                    y_batch, \
                    np.expand_dims(r, axis = 0), \
                    s_batch)
    break

(1, 1, 32, 32)


/home/eirasf/miniconda3/envs/xai-anna/lib/python3.11/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/eirasf/miniconda3/envs/xai-anna/lib/python3.11/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:64: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  warnings.warn(


IndexError: index 1946 is out of bounds for axis 0 with size 1024